# 進階查詢

列出同一標題的全部價格

In [3]:
import pymysql
import os
from dotenv import load_dotenv
from collections import defaultdict

# 載入設定
load_dotenv()
db_config = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME")
}

# 🔸 查詢條件（可依需求修改）
search_keyword = "BLUE BAY 倍力 Sense"
# filter_keyword_in_title = "雪肌粹"
filter_keyword_in_title = "3種魚"

# 連線並查詢
connection = pymysql.connect(
    host=db_config["host"],
    port=db_config["port"],
    user=db_config["user"],
    password=db_config["password"],
    database=db_config["database"],
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with connection:
    with connection.cursor() as cursor:
        sql = """
        SELECT title, price
        FROM coupang_products
        WHERE search_keyword LIKE %s AND title LIKE %s
        ORDER BY title
        """
        cursor.execute(
            sql,
            (f"%{search_keyword}%", f"%{filter_keyword_in_title}%")
        )
        rows = cursor.fetchall()

# 分組：依 title 對應多個不同價格
grouped = defaultdict(set)
for row in rows:
    grouped[row['title']].add(row['price'])

# 輸出每個標題對應的價格列表
print(f"📊 搜尋條件：search_keyword 含『{search_keyword}』，title 含『{filter_keyword_in_title}』\n")
for title, prices in grouped.items():
    print(f"🔹 標題: {title}")
    for price in sorted(prices):
        print(f"   - 價格: {price}")
    print("-" * 50)


📊 搜尋條件：search_keyword 含『BLUE BAY 倍力 Sense』，title 含『3種魚』

🔹 標題: BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 1袋
   - 價格: $345
--------------------------------------------------
🔹 標題: BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 2袋
   - 價格: $790
--------------------------------------------------
🔹 標題: BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 3袋
   - 價格: $1,235
--------------------------------------------------
🔹 標題: BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 4袋
   - 價格: $1,680
--------------------------------------------------


列出價格有兩種以上者

In [12]:
import pymysql
import os
from dotenv import load_dotenv
from collections import defaultdict

# 載入設定
load_dotenv()
db_config = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 3306)),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME")
}

# 查詢條件
search_keyword = "BLUE BAY 倍力 Sense"
# 進階篩選關鍵字列表，邏輯條件是交集出現在 title 中
filter_keywords_in_title = ["3種魚"]

# SQL 查詢，不含進階篩選
sql = """
    SELECT title, price, timestamp
    FROM coupang_products
    WHERE search_keyword LIKE %s
    ORDER BY title, timestamp
"""

# 建立連線並查詢
connection = pymysql.connect(
    host=db_config["host"],
    port=db_config["port"],
    user=db_config["user"],
    password=db_config["password"],
    database=db_config["database"],
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with connection:
    with connection.cursor() as cursor:
        cursor.execute(sql, (f"%{search_keyword}%",))
        rows = cursor.fetchall()

# 進階篩選：title 同時包含所有關鍵字
def is_match_advanced(title: str, keywords: list[str]) -> bool:
    return all(kw in title for kw in keywords)

# 過濾 rows
if filter_keywords_in_title:
    rows = [r for r in rows if is_match_advanced(r['title'], filter_keywords_in_title)]

# 分組：依 title 對應多筆 (價格，時間)
grouped = defaultdict(list)
for row in rows:
    grouped[row['title']].append((row['price'], row['timestamp']))

# 僅保留有兩種以上價格的 title，根據價格種類去重
filtered_grouped = {
    title: entries
    for title, entries in grouped.items()
    if len(set(price for price, _ in entries)) >= 2
}

# 輸出結果
print(f"搜尋條件：search_keyword 含『{search_keyword}』")
if filter_keywords_in_title:
    print(
        "進階條件：title 同時包含"
        f"『{'、'.join(filter_keywords_in_title)}』\n"
    )
# 多筆價格資訊
if not filtered_grouped:
    print("⚠️ 查無重複價格變化的項目")
else:
    for title, records in filtered_grouped.items():
        print(f"🔹 標題：{title}")
        for price, timestamp in sorted(records, key=lambda x: x[1]):
            print(f"   - 價格：{price}（時間：{timestamp}）")
        print("-" * 60)


搜尋條件：search_keyword 含『BLUE BAY 倍力 Sense』
進階條件：title 同時包含『3種魚』

🔹 標題：BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 1袋
   - 價格：$345（時間：2025-05-12 02:20:02）
   - 價格：$345（時間：2025-05-12 02:23:24）
   - 價格：$345（時間：2025-05-12 02:26:48）
   - 價格：$345（時間：2025-05-12 02:27:55）
   - 價格：$345（時間：2025-05-12 02:37:14）
   - 價格：$345（時間：2025-05-12 02:37:31）
   - 價格：$345（時間：2025-05-12 02:37:32）
   - 價格：$279（時間：2025-05-17 19:48:13）
   - 價格：$341（時間：2025-05-17 20:06:34）
------------------------------------------------------------
🔹 標題：BLUE BAY 倍力 Sense 全護低敏貓飼料 美膚爆毛配方 3種魚 + 台灣鱉蛋, 皮膚/毛髮, 1.5kg, 2袋
   - 價格：$790（時間：2025-05-12 02:20:02）
   - 價格：$790（時間：2025-05-12 02:23:24）
   - 價格：$790（時間：2025-05-12 02:26:48）
   - 價格：$790（時間：2025-05-12 02:27:55）
   - 價格：$790（時間：2025-05-12 02:37:14）
   - 價格：$790（時間：2025-05-12 02:37:31）
   - 價格：$790（時間：2025-05-12 02:37:32）
   - 價格：$658（時間：2025-05-17 19:48:13）
   - 價格：$708（時間：2025-05-17 20:06:34）
------------------------------------------------------------
🔹 標題：BLUE BAY 倍